In [1]:
## Imports
import os
import numpy as np                    
import matplotlib.pyplot as plt
import seaborn as sns
from Lab_Analyses.Spine_Analysis_v2 import spine_utilities as s_utils

sns.set()
sns.set_style("ticks")

In [84]:
## Specify data to load
mouse_id = "JW153"
days = ["Early"]
fov_type = "apical"

# Load the data
full_data = s_utils.load_spine_datasets(mouse_id, days, fov_type)

In [85]:
# Grab relevant data
data = full_data["FOV2_apical"]["Early"]

In [25]:
data.__dict__.keys()

dict_keys(['mouse_id', 'session', 'date', 'imaging_parameters', 'time', 'lever_force_resample', 'lever_force_smooth', 'lever_velocity_envelope', 'lever_active', 'rewarded_movement_force', 'rewarded_movement_binary', 'binary_cue', 'reward_delivery', 'punish_delivery', 'spine_flags', 'spine_groupings', 'spine_positions', 'spine_volume', 'corrected_spine_volume', 'spine_GluSnFr_dFoF', 'spine_GluSnFr_processed_dFoF', 'spine_GluSnFr_activity', 'spine_GluSnFr_floored', 'spine_calcium_dFoF', 'spine_calcium_processed_dFoF', 'spine_calcium_activity', 'spine_calcium_floored', 'movement_spines', 'reward_movement_spines', 'silent_spines', 'reward_silent_spines', 'dendrite_length', 'dendrite_calcium_dFoF', 'dendrite_calcium_processed_dFoF', 'dendrite_calcium_activity', 'dendrite_calcium_floored', 'movement_dendrites', 'reward_movement_dendrites', 'silent_dendrites', 'reward_silent_dendrites', 'poly_dendrite_positions', 'poly_dendrite_calcium_dFoF', 'poly_dendrite_calcium_processed_dFoF', 'followup_

In [ ]:
data.imaging_parameters

In [86]:
# Get activity related data
spine_activity = data.spine_GluSnFr_activity
spine_dFoF = data.spine_GluSnFr_processed_dFoF
dendrite_activity = data.dendrite_calcium_activity
dendrite_dFoF = data.dendrite_calcium_processed_dFoF
spine_calcium = data.spine_calcium_processed_dFoF

In [87]:
# Mask active and inactive portions
def get_active_inactive(dFoF, active):
    inactive_mask = (active == 1)
    active_mask = (active == 0)
    active_dFoF = np.copy(dFoF)
    inactive_dFoF = np.copy(dFoF)
    active_dFoF[active_mask] = np.nan
    inactive_dFoF[inactive_mask] = np.nan
    
    return active_dFoF, inactive_dFoF

In [88]:
spine_active = np.zeros(spine_activity.shape) * np.nan
spine_inactive = np.zeros(spine_activity.shape) * np.nan
dend_active = np.zeros(dendrite_activity.shape) * np.nan
dend_inactive = np.zeros(dendrite_activity.shape) * np.nan

for i in range(spine_activity.shape[1]):
    s_active, s_inactive = get_active_inactive(spine_dFoF[:, i], spine_activity[:, i])
    d_active, d_inactive = get_active_inactive(dendrite_dFoF[:, i], dendrite_activity[:, i])
    spine_active[:, i] = s_active
    spine_inactive[:, i] = s_inactive
    dend_active[:, i] = d_active
    dend_inactive[:, i] = d_inactive

In [29]:
# Explore the traces
%matplotlib qt

for i in range(spine_activity.shape[1]):
    plt.figure()
    plt.plot(spine_inactive[:, i], color="black")
    plt.plot(spine_active[:, i], color="blue")
    plt.ylim(bottom=-0.5, top=1.5)

C:\Users\Jake\AppData\Local\Temp\ipykernel_24020\385542645.py:5: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  plt.figure()


In [91]:
# Group plot
%matplotlib qt
save_path = r"C:\Users\Jake\Desktop\Figures\Examples\Example_traces"

x = np.arange(len(spine_active)) / 60
fig, ax = plt.subplot_mosaic("""A""",figsize=(7,10))
title = "JW153_FOV2_Early_Glut_Calcium_Traces"
fig.suptitle(title)
ax["A"].plot(x[:], dend_inactive[:,0], color="red", alpha=0.3, linewidth=.8)
ax["A"].plot(x[:], dend_active[:,0], color="red", linewidth=.8)

for  i in reversed(range(spine_activity.shape[1])):
#for j, i in enumerate(reversed([5,9,1])):
    if np.isnan(spine_active[:, i]).all():
        continue
    print(i)
    #coactivity = dendrite_activity[:, 0] * spine_activity[:, i]
    ax["A"].plot(x[:], spine_inactive[:, i] + i +1.1, color="blue", alpha=0.3, linewidth=.8)
    ax["A"].plot(x[:], spine_active[:, i] + i + 1.1, color="blue", linewidth=.8)
    ax["A"].plot(x[:], spine_calcium[:, i] + i + 1.1, color="black", linewidth=.8)

fname = os.path.join(save_path, title)
#fname = os.path.join(save_path, "Spine_10_traces")
#fig.savefig(fname + ".pdf")

23
22
21
20
19
18
17
16
15
14
13
12
11
10
9
8
7
6
5
4
3
2
1
0


In [ ]:
fig, ax = plt.subplot_mosaic("""A""",figsize=(7,10))
x = np.arange(len(spine_active)) / 60
ax["A"].plot(x[5100:6300], dendrite_activity[5100:6300, 0], color="red", alpha=1, linewidth=.8)
for i in reversed(range(30,37)):
    ax["A"].plot(x[5100:6300], spine_activity[5100:6300, i] + i +1.1, color="black", alpha=1, linewidth=.8)
    
fname = os.path.join(save_path, "Example_coincidental_activity_binary")
fig.savefig(fname + ".pdf")

In [ ]:
## Elim spine examples
from numpy import genfromtxt
control_dFoF = genfromtxt(r'Z:\People\Jake\Ctl_dFoF.csv', delimiter=',').T
control_activity = genfromtxt(r"Z:\People\Jake\Ctl_Active.csv", delimiter=",").T
elim_dFoF = genfromtxt(r"Z:\People\Jake\Elim_dFoF.csv", delimiter=",").T
elim_activity = genfromtxt(r"Z:\People\Jake\Elim_Active.csv", delimiter=",").T

In [ ]:
ctl_active = np.zeros(control_activity.shape) * np.nan
ctl_inactive = np.zeros(control_activity.shape) * np.nan
elim_active = np.zeros(elim_activity.shape) * np.nan
elim_inactive = np.zeros(elim_activity.shape) * np.nan

for i in range(control_activity.shape[1]):
    s_active, s_inactive = get_active_inactive(control_dFoF[:, i], control_activity[:, i])
    ctl_active[:, i] = s_active
    ctl_inactive[:, i] = s_inactive

for i in range(elim_activity.shape[1]):
    s_active, s_inactive = get_active_inactive(elim_dFoF[:, i], elim_activity[:, i])
    elim_active[:, i] = s_active
    elim_inactive[:, i] = s_inactive

In [ ]:
# Group plot
%matplotlib qt
save_path = r"C:\Users\Jake\Desktop\Figures"

x = np.arange(len(ctl_active)) / 60
fig, ax = plt.subplot_mosaic("""A""",figsize=(7,10))
title = "Control Spine Examples (elim proj.)"
fig.suptitle(title)


for i in reversed(range(control_activity.shape[1])):
    if np.isnan(ctl_active[:, i]).all():
        continue
    print(i)
    ax["A"].plot(x[:], ctl_inactive[:, i] + i +1.1, color="black", alpha=0.3, linewidth=.8)
    ax["A"].plot(x[:], ctl_active[:, i] + i + 1.1, color="black", linewidth=.8)

fname = os.path.join(save_path, title)
fname = os.path.join(save_path, "Ctl_example_traces")
fig.savefig(fname + ".pdf")

In [ ]:
# Group plot
%matplotlib qt
save_path = r"C:\Users\Jake\Desktop\Figures"

x = np.arange(len(elim_active)) / 60
fig, ax = plt.subplot_mosaic("""A""",figsize=(7,10))
title = "Elim. Spine Examples (elim proj.)"
fig.suptitle(title)


for i in reversed(range(elim_activity.shape[1])):
    if np.isnan(elim_active[:, i]).all():
        continue
    print(i)
    ax["A"].plot(x[:], elim_inactive[:, i] + i +1.1, color="red", alpha=0.3, linewidth=.8)
    ax["A"].plot(x[:], elim_active[:, i] + i + 1.1, color="red", linewidth=.8)

fname = os.path.join(save_path, title)
fname = os.path.join(save_path, "Elim_example_traces")
fig.savefig(fname + ".pdf")